Purpose of this notebook is to get full text articles from PMC

In [17]:
# imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os, json
import requests
import time

import sys
sys.path.append('../pubmed_rag')

from bioc import get_biocjson, passages_to_df

# import nltk
# from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')

#from spellchecker import SpellChecker

In [9]:
df = pd.read_csv('data/test_pmid_list.csv', header=None)
df.head()

pmids = df[0].astype(str).to_list()

### Current status 

Getting full texts via biocjson from [Pubtator](https://www.ncbi.nlm.nih.gov/research/pubtator3/api) instead of [BioC API for PMC](https://www.ncbi.nlm.nih.gov/research/bionlp/APIs/BioC-PMC/)

Why?? 
- I only have pmids currently, and Pubtator automatically links pmid to pmcid to return full text if it is available, otherwise returns abstract.
- BioC API for PMC is not compatible with pmid id even though it says it does

Bioc files
- it looks like each 'passage' is a sentence
- and the passages have annotations 
- additionally, relevant to us, the passage has metadata such as what section its from


Steps
- config
    - provide list of pmids 
    - output path
- retrieve biocjson (full text or abstract) and save to output path
- for each json parse for text + section

Vertex database of sentences or sections?
Option 1:
- embedd each sentence using [PubMedBERT now called BiomedBERT](https://huggingface.co/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext)
- keeping metadata

Option 2:
- collapse the sentences into one section 
- embed each section using same model




In [18]:
# Example usage
sample = pmids[:15]


In [19]:
for each in sample:
    
    result = get_biocjson(each, 'biocjson')

    if result is not None:
        passages_to_df(result, 'biocjson')

    time.sleep(2)

In [26]:
# cleaning?
# read in example
df_test = pd.read_csv('biocjson/df_26158728.csv')
# lower case section names
df_test['section'] = df_test['section'].str.lower().str.strip()
# pmids to object
df_test['pmid'] = df_test['pmid'].astype(str)
df_test['date'] = pd.to_datetime(df_test['date'])
# check
df_test.head()

,index,section,sentence,pmid,pmcid,date,authors,journal
0,0,title,Heterogeneous Network Edge Prediction: A Data ...,26158728,PMC4497619,2015-07-09 00:00:00+00:00,Himmelstein DS and Baranzini SE,PLoS Comput Biol
1,1,abstract,The first decade of Genome Wide Association St...,26158728,PMC4497619,2015-07-09 00:00:00+00:00,Himmelstein DS and Baranzini SE,PLoS Comput Biol
2,2,abstract,Author Summary,26158728,PMC4497619,2015-07-09 00:00:00+00:00,Himmelstein DS and Baranzini SE,PLoS Comput Biol
3,3,abstract,"For complex human diseases, identifying the ge...",26158728,PMC4497619,2015-07-09 00:00:00+00:00,Himmelstein DS and Baranzini SE,PLoS Comput Biol
4,4,intro,Introduction,26158728,PMC4497619,2015-07-09 00:00:00+00:00,Himmelstein DS and Baranzini SE,PLoS Comput Biol


In [81]:
def annotate_pmids(pmids):
    # Define the PubTator API URL
    pubtator_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocxml"

    # Join the PMIDs into a comma-separated string
    pmid_str = ','.join(pmids)

    # Send a GET request to the API with the list of PMIDs
    response = requests.get(pubtator_url, params={"pmids": pmid_str, "full": True})

    # Check if the request was successful
    if response.status_code == 200:
        with open(f"PMC4448285.xml", "w", encoding="utf-8") as f:
            f.write(response.text)
        print(f"Downloaded full text XML as 4448285.xml")
    # if response.status_code == 200:
        #return response.json()  # Return the response in JSON format
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")

# Example usage
pmids = ["25133803", "25971816"]
annotations = annotate_pmids(pmids)

# Display the annotated results
import pprint
pprint.pprint(annotations)


Downloaded full text XML as 4448285.xml
None


In [ ]:
import xml.etree.ElementTree as ET

def extract_full_text_from_bioc(xml_file):
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Initialize a list to store text passages
    full_text = []

    # Iterate through passages in the document
    for document in root.findall(".//document"):
        for passage in document.findall("passage"):
            # Extract the text content
            text = passage.find("text").text
            if text:
                full_text.append(text.strip())

    # Join all passages into a single string with new lines
    #return "\n\n".join(full_text)
    return full_text
